Primero importo las librerias

In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split
usuario=2
import os
from matplotlib.image import imread
import random
import numpy as np
import glob as glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

  Using cached ultralytics-8.2.14-py3-none-any.whl (756 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using ca

In [ ]:
!python '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.20 --source '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/Test/images' --name '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO' --save-txt --save-conf

#!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.20 --source '/content/drive/MyDrive/Proyecto final/Test/images' --name '/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/weights/best.pt'], source=/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/Test/images, data=drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.2, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
ERROR: pip's depen

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.8414, Recall: 0.8301, F1-Score: 0.8357

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.8000, Recall: 0.8889, F1-Score: 0.8421
Image: 53994_6-8-1-1.txt, Precision: 0.7826, Recall: 0.7500, F1-Score: 0.7660
Image: 5702_29-1-1-3.txt, Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000
Image: 53819_3-10-1-4.txt, Precision: 0.8710, Recall: 0.7941, F1-Score: 0.8308
Image: 53853_37-18-1-1.txt, Precision: 0.5333, Recall: 0.5333, F1-Score: 0.5333
Image: 53819_7-12-1-4.txt, Precision: 0.6970, Recall: 0.8214, F1-Score: 0.7541
Image: 3153_39-2-1-2.txt, Precision: 0.9333, Recall: 0.8750, F1-Score: 0.9032
Image: 34417_15-3-1-1.txt, Precision: 0.9750, Recall: 1.0000, F1-Score: 0.9873
Image: 53823_2-4-2-1.txt, Precision: 0.7727, Recall: 0.8947, F1-Score: 0.8293
Image: 37186_8-9-1-5.txt, Precision: 0.8250, Recall: 0.8250, F1-Score: 0.8250
Image: 38028_1-6-11-12.txt, Precision: 0.2105, Recall: 0.2222, F1-Score: 0.2162
Image: 6672_42-4-1-6.txt, Precision: 1.000

In [1]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2/labels'

# draw_yolo_annotations(image_folder, label_folder)

In [2]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Test/labels'

# draw_yolo_annotations(image_folder, label_folder)

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 3175_1-5-1-3.txt
Predicted objects: 20, Real objects: 18
Image: 53994_6-8-1-1.txt
Predicted objects: 23, Real objects: 24
Image: 5702_29-1-1-3.txt
Predicted objects: 17, Real objects: 17
Image: 53819_3-10-1-4.txt
Predicted objects: 31, Real objects: 34
Image: 53853_37-18-1-1.txt
Predicted objects: 15, Real objects: 15
Image: 53819_7-12-1-4.txt
Predicted objects: 33, Real objects: 28
Image: 3153_39-2-1-2.txt
Predicted objects: 15, Real objects: 16
Image: 34417_15-3-1-1.txt
Predicted objects: 40, Real objects: 39
Image: 53823_2-4-2-1.txt
Predicted objects: 22, Real objects: 19
Image: 37186_8-9-1-5.txt
Predicted objects: 40, Real objects: 40
Image: 38028_1-6-11-12.txt
Predicted objects: 19, Real objects: 18
Image: 6672_42-4-1-6.txt
Predicted objects: 15, Real objects: 15
Image: 37807_1-7-15-4.txt
Predicted objects: 31, Real objects: 24
Image: 3140_80-9-1-2.txt
Predicted objects: 10, Real objects: 11
Image: 3181_76-6-1-1.txt
Predicted objects: 16, Real objects: 15
Image: 7307_22-2-1

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2/labels'
def draw_yolo_annotations_ordered_by_precision(image_folder, label_folder, metrics_per_image):
    # Ordena las métricas por imagen según la precisión de menor a mayor
    metrics_sorted = sorted(metrics_per_image, key=lambda x: x['Precision'])

    # Procesa cada imagen en el orden de las métricas ordenadas
    for metric in metrics_sorted:
        img_file = metric['Filename']
        if img_file.endswith(('.png', '.jpg', '.jpeg')):
            base_name = os.path.splitext(img_file)[0]
            label_file = f"{base_name}.txt"

            # Verifica que el archivo de etiquetas exista
            label_path = os.path.join(label_folder, label_file)
            if os.path.isfile(label_path):
                image_path = os.path.join(image_folder, img_file)

                # Carga la imagen
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error cargando la imagen {image_path}")
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Leer el archivo .txt con anotaciones YOLO y dibujarlas
                with open(label_path, 'r') as file:
                    for line in file:
                        parts = line.strip().split()
                        x_center, y_center, w, h = map(float, parts[1:5])
                        x_center *= image.shape[1]
                        y_center *= image.shape[0]
                        w *= image.shape[1]
                        h *= image.shape[0]
                        x1 = int(x_center - w / 2)
                        y1 = int(y_center - h / 2)
                        x2 = int(x_center + w / 2)
                        y2 = int(y_center + h / 2)

                        # Dibujar el rectángulo sobre la imagen
                        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Mostrar la imagen con el título incluyendo la precisión
                plt.figure(figsize=(10, 10))
                plt.imshow(image)
                plt.title(f"{img_file} - Precision: {metric['Precision']:.4f}")
                plt.show()
            else:
                print(f"No se encontró archivo de etiquetas para {img_file}")


#raw_yolo_annotations(image_folder, label_folder)
draw_yolo_annotations_ordered_by_precision(image_folder, label_folder, metrics_per_image)


Analizamos labels predichos país, fecha y tags

In [ ]:
import os
import pandas as pd

# Definir las rutas de las carpetas y archivos
label_folder = '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2/labels' #labels predichas
excel_path = '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/layouts_todo.csv'
image_folder = '/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/TrainingVacasVaqueras/MODELOfinal/TESTEO2'


# Leer el archivo Excel
layouts_todo=pd.read_csv(excel_path, encoding='latin1')

# Obtener los layout_id de cada archivo de etiqueta
label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]
file_layout_ids = [f.split('_')[0] for f in label_files]

# Crear una lista para almacenar los datos
data = []

# Iterar sobre los archivos de etiqueta y obtener los valores correspondientes
for file, layout_id in zip(label_files, file_layout_ids):
    country = layouts_todo.loc[layouts_todo['layout_id'] == int(layout_id), 'country'].values[0]
    flight_date = layouts_todo.loc[layouts_todo['layout_id'] == int(layout_id), 'flight_date'].values[0]
    relevant_tags = layouts_todo.loc[layouts_todo['layout_id'] == int(layout_id), 'relevant (new) tags'].values[0]
    layout_id = int(layout_id)  # Convertir layout_id a entero
    data.append([file, layout_id, country, flight_date, relevant_tags])

# Crear el DataFrame
df = pd.DataFrame(data, columns=['label_file','layout_id', 'country', 'flight_date', 'relevant_tags'])

# Mostrar el DataFrame
print(df)



             label_file  layout_id    country      flight_date relevant_tags
0    31255_14-3-1-1.txt      31255  Argentina   2/4/2021 14:12        4,6,11
1    31255_16-3-1-2.txt      31255  Argentina   2/4/2021 14:12        4,6,11
2     31396_1-1-1-4.txt      31396  Argentina  1/19/2021 10:30           4,5
3    31396_15-1-1-3.txt      31396  Argentina  1/19/2021 10:30           4,5
4     31396_3-8-1-3.txt      31396  Argentina  1/19/2021 10:30           4,5
..                  ...        ...        ...              ...           ...
391   7307_10-7-1-1.txt       7307     Brazil   3/8/2019 14:27         2,4,5
392   7307_22-2-1-4.txt       7307     Brazil   3/8/2019 14:27         2,4,5
393   7307_22-4-1-1.txt       7307     Brazil   3/8/2019 14:27         2,4,5
394   7307_29-5-1-1.txt       7307     Brazil   3/8/2019 14:27         2,4,5
395   7307_33-8-1-1.txt       7307     Brazil   3/8/2019 14:27         2,4,5

[396 rows x 5 columns]


Funciones para evaluar resultados comparando grupos

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

def draw_yolo_annotations(image_folder, label_folder, label_files):
    # Procesar cada archivo de imagen
    for label_file in label_files:
        img_file = label_file.replace('.txt', '.png')  # Obtiene el nombre del archivo de imagen correspondiente
        print('nombre archivo imágen',img_file)
        image_path = os.path.join(image_folder, img_file)
        print('path archivo imágen',image_path)
        label_path = os.path.join(label_folder, label_file)

        # Verifica que el archivo de imagen exista
        if os.path.exists(image_path):
            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de imagen para {label_file}")
def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0
# Función para cargar las etiquetas solo para la variable actual
def load_labels(label_folder, label_files):
    labels = {}
    for label_file in label_files:
        with open(os.path.join(label_folder, label_file), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[label_file] = boxes
    return labels

# Función para calcular métricas solo para la variable actual
def calculate_metrics(pred_labels, real_labels, label_files, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for label_file in label_files:
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        real_boxes = real_labels.get(label_file, [])
        pred_boxes = pred_labels.get(label_file, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': label_file,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics


Experimentación

In [4]:
def filtrar_por_pais(df, country):
    return df[df['country'] == country]

def filtrar_por_hora_vuelo(df, flight_hour):
    # Convertir la hora de vuelo al formato deseado, por ejemplo, extrayendo la hora de la fecha de vuelo
    return df[df['flight_date'].dt.hour == flight_hour]

def filtrar_por_tag(df, tag):
    return df[df['relevant_tags'].apply(lambda tags: str(tag) in str(tags).split(',') if pd.notnull(tags) else False)]

# Crear una nueva columna 'flight_hour' con las horas de vuelo
df['flight_date'] = pd.to_datetime(df['flight_date'])
df['flight_hour'] = df['flight_date'].dt.hour

variables = ['country', 'flight_hour', 'relevant_tags']  # Lista de variables a comparar

for variable in variables:
    if variable == 'country':
        valores_unicos = df[variable].unique()
    elif variable == 'flight_hour':
        valores_unicos = df['flight_hour'].unique()  # Utilizar la nueva columna 'flight_hour'
    elif variable == 'relevant_tags':
        valores_unicos = [1,2,3,4,5,6,7,8,9,10,11]

    for valor in valores_unicos:
        if variable == 'country':
            df_filtrado = filtrar_por_pais(df, valor)
        elif variable == 'flight_hour':
            df_filtrado = filtrar_por_hora_vuelo(df, valor)
        elif variable == 'relevant_tags':
           df_filtrado = filtrar_por_tag(df, valor)

        # Realiza las operaciones adicionales con el DataFrame filtrado, por ejemplo, calcular métricas
      # Seleccionar aleatoriamente 5 label_files del DataFrame filtrado
        random_files = random.sample(df_filtrado['label_file'].tolist(), min(5, len(df_filtrado)))

        # Ejecutar las funciones para las etiquetas e imágenes de los archivos seleccionados aleatoriamente
        draw_yolo_annotations(image_folder, label_folder, random_files)  # Llama la función para las etiquetas e imágenes correspondientes a los archivos seleccionados

        #layout_ids_variable =  df['layout_id'].astype(str).tolist()
        label_files_variable = df_filtrado['label_file']#df[f for f in os.listdir(label_folder) if f.split('_')[0] in layout_ids_variable]
        # Cargar las etiquetas predichas y reales solo para el país actual
        pred_labels = load_labels(label_folder, label_files_variable)
        real_labels = load_labels('/content/drive/MyDrive/2024/Primer Cuatrimestre/PF /Proyecto final/Test/labels',label_files_variable)

        # Calcula métricas solo para el vakor indicado
        metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels, label_files_variable)

        # Imprime las métricas generales
        print("Métricas generales -",variable,":",valor)
        print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
        print()
